In [8]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Data Tidying and Cleaning Lab
## Reading, tidying and cleaning data. Preparing data for exploration, mining, analysis and learning

In this lab, you'll be working with the Coffee Quality Index dataset, located [here](https://www.kaggle.com/datasets/volpatto/coffee-quality-database-from-cqi). For convenience (and to save trouble in case you can't download files, or someone uploads a newer version), I've provided the dataset in the `data/` folder. The metadata (description) is at the Kaggle link. For this lab, you'll only need `merged_data_cleaned.csv`, as it is the concatenation of the other two datasets.

In this (and the following labs), you'll get several questions and problems. Do your analysis, describe it, use any tools and plots you wish, and answer. You can create any amount of cells you'd like.

Sometimes, the answers will not be unique, and they will depend on how you decide to approach and solve the problem. This is usual - we're doing science after all!

It's a good idea to save your clean dataset after all the work you've done to it.

### Problem 1. Read the dataset (1 point)
This should be self-explanatory. The first column is the index.

In [3]:
# Read the coffee dataset
# The first column is the index as mentioned in the problem
df = pd.read_csv('data/merged_data_cleaned.csv', index_col=0)

# Display basic information about the dataset
print("Dataset successfully loaded!")
print(f"Shape: {df.shape}")
print(f"Number of rows: {df.shape[0]}")
print(f"Number of columns: {df.shape[1]}")

# Display column names
print("\nColumn names:")
print(df.columns.tolist())

# Display first few rows
print("\nFirst 5 rows:")
print(df.head())

# Display basic info about the dataset
print("\nDataset info:")
print(df.info())

# Display basic statistics
print("\nBasic statistics for numerical columns:")
print(df.describe())

Dataset successfully loaded!
Shape: (1339, 43)
Number of rows: 1339
Number of columns: 43

Column names:
['Species', 'Owner', 'Country.of.Origin', 'Farm.Name', 'Lot.Number', 'Mill', 'ICO.Number', 'Company', 'Altitude', 'Region', 'Producer', 'Number.of.Bags', 'Bag.Weight', 'In.Country.Partner', 'Harvest.Year', 'Grading.Date', 'Owner.1', 'Variety', 'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points', 'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Quakers', 'Color', 'Category.Two.Defects', 'Expiration', 'Certification.Body', 'Certification.Address', 'Certification.Contact', 'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']

First 5 rows:
   Species                     Owner Country.of.Origin  \
0  Arabica                 metad plc          Ethiopia   
1  Arabica                 metad plc          Ethiopia   
2  Arabica  grounds for health admin     

### Problem 2. Observations and features (1 point)
How many observations are there? How many features? Which features are numerical, and which are categorical?

**Note:** Think about the _meaning_, not the data types. The dataset hasn't been thoroughly cleaned.

In [8]:

# Continue from Problem 1 - assuming df is already loaded
# df = pd.read_csv('merged_data_cleaned.csv', index_col=0)

print("=== OBSERVATIONS AND FEATURES ANALYSIS ===\n")

# Number of observations and features
print(f"Number of observations (rows): {df.shape[0]}")
print(f"Number of features (columns): {df.shape[1]}")

print("\n" + "="*50)

# Let's examine each column to determine if it's numerical or categorical based on MEANING
print("\nCOLUMN ANALYSIS (by meaning, not just data type):\n")

# Define categorical and numerical features based on meaning
categorical_features = []
numerical_features = []

# Examine each column
for col in df.columns:
    print(f"Column: {col}")
    print(f"  Data type: {df[col].dtype}")
    print(f"  Unique values: {df[col].nunique()}")
    print(f"  Sample values: {df[col].dropna().head(3).tolist()}")
    
    # Determine if categorical or numerical based on meaning
    if col in ['Species', 'Owner', 'Country.of.Origin', 'Farm.Name', 'Lot.Number', 
               'Mill', 'ICO.Number', 'Company', 'Region', 'Producer', 'Bag.Weight',
               'In.Country.Partner', 'Grading.Date', 'Owner.1', 'Variety', 
               'Processing.Method', 'Color', 'Expiration', 'Certification.Body',
               'Certification.Address', 'Certification.Contact', 'unit_of_measurement']:
        categorical_features.append(col)
        print(f"  → CATEGORICAL (represents categories/labels)")
    
    elif col in ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
                 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points', 
                 'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 
                 'Quakers', 'Category.Two.Defects', 'altitude_low_meters',
                 'altitude_high_meters', 'altitude_mean_meters', 'Number.of.Bags']:
        numerical_features.append(col)
        print(f"  → NUMERICAL (represents measurable quantities)")
    
    # Special cases that need closer inspection
    elif col in ['Altitude', 'Harvest.Year']:
        # Altitude is stored as text ranges but represents numerical values
        # Harvest.Year should be numerical but might have data quality issues
        if col == 'Altitude':
            categorical_features.append(col)  # Currently stored as text ranges
            print(f"  → CATEGORICAL (currently stored as text ranges, but conceptually numerical)")
        else:  # Harvest.Year
            numerical_features.append(col)
            print(f"  → NUMERICAL (represents year, but may have data quality issues)")
    
    print()

print("="*50)
print(f"\nSUMMARY:")
print(f"Total observations: {df.shape[0]}")
print(f"Total features: {df.shape[1]}")
print(f"Categorical features: {len(categorical_features)}")
print(f"Numerical features: {len(numerical_features)}")

print(f"\nCATEGORICAL FEATURES ({len(categorical_features)}):")
for i, feat in enumerate(categorical_features, 1):
    print(f"{i:2d}. {feat}")

print(f"\nNUMERICAL FEATURES ({len(numerical_features)}):")
for i, feat in enumerate(numerical_features, 1):
    print(f"{i:2d}. {feat}")

print(f"\n" + "="*50)
print("IMPORTANT NOTES:")
print("- Classification based on MEANING, not just data types")
print("- Some 'object' columns like Harvest.Year contain numerical data")
print("- Some numerical columns might have data quality issues")
print("- Altitude is stored as text ranges but represents numerical concept")
print("- The dataset needs cleaning for proper analysis")

# Let's also check for potential data quality issues
print(f"\n" + "="*50)
print("DATA QUALITY INSIGHTS:")

# Check Harvest.Year for non-numeric values
print(f"\nHarvest.Year unique values: {sorted(df['Harvest.Year'].dropna().unique())}")

# Check missing values
print(f"\nMissing values per column:")
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0].sort_values(ascending=False)
for col, count in missing_counts.items():
    print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")

if len(missing_counts) == 0:
    print("  No missing values found")

=== OBSERVATIONS AND FEATURES ANALYSIS ===

Number of observations (rows): 1339
Number of features (columns): 43


COLUMN ANALYSIS (by meaning, not just data type):

Column: Species
  Data type: object
  Unique values: 2
  Sample values: ['Arabica', 'Arabica', 'Arabica']
  → CATEGORICAL (represents categories/labels)

Column: Owner
  Data type: object
  Unique values: 315
  Sample values: ['metad plc', 'metad plc', 'grounds for health admin']
  → CATEGORICAL (represents categories/labels)

Column: Country.of.Origin
  Data type: object
  Unique values: 36
  Sample values: ['Ethiopia', 'Ethiopia', 'Guatemala']
  → CATEGORICAL (represents categories/labels)

Column: Farm.Name
  Data type: object
  Unique values: 571
  Sample values: ['metad plc', 'metad plc', 'san marcos barrancas "san cristobal cuch']
  → CATEGORICAL (represents categories/labels)

Column: Lot.Number
  Data type: object
  Unique values: 227
  Sample values: ['YNC-06114', '102', 'Tsoustructive 2015 Sumatra Typica']
  → CA

### Problem 3. Column manipulation (1 point)
Make the column names more Pythonic (which helps with the quality and... aesthetics). Convert column names to `snake_case`, i.e. `species`, `country_of_origin`, `ico_number`, etc. Try to not do it manually.

In [4]:
# Continue from previous problems - assuming df is already loaded
# df = pd.read_csv('merged_data_cleaned.csv', index_col=0)

print("=== COLUMN MANIPULATION TO SNAKE_CASE ===\n")

# Display original column names
print("ORIGINAL COLUMN NAMES:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal columns: {len(df.columns)}")

# Function to convert to snake_case
def to_snake_case(name):
    """
    Convert column name to snake_case format
    """
    # Handle special cases and clean up
    name = str(name).strip()
    
    # Replace dots with underscores
    name = name.replace('.', '_')
    
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    
    # Replace hyphens with underscores
    name = name.replace('-', '_')
    
    # Insert underscore before uppercase letters (for camelCase)
    name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', name)
    
    # Convert to lowercase
    name = name.lower()
    
    # Remove multiple consecutive underscores
    name = re.sub(r'_+', '_', name)
    
    # Remove leading/trailing underscores
    name = name.strip('_')
    
    return name

# Create mapping of old to new column names
column_mapping = {}
for col in df.columns:
    new_name = to_snake_case(col)
    column_mapping[col] = new_name

print("\n" + "="*60)
print("COLUMN NAME TRANSFORMATION:")
print("="*60)

for old_name, new_name in column_mapping.items():
    print(f"{old_name:<25} → {new_name}")

# Apply the transformation
df_renamed = df.rename(columns=column_mapping)

print(f"\n" + "="*60)
print("NEW COLUMN NAMES (snake_case):")
for i, col in enumerate(df_renamed.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\nTotal columns: {len(df_renamed.columns)}")

# Verify the transformation worked
print(f"\n" + "="*60)
print("VERIFICATION:")
print(f"Original shape: {df.shape}")
print(f"New shape: {df_renamed.shape}")
print(f"Data preserved: {df.equals(df_renamed.rename(columns={v: k for k, v in column_mapping.items()}))}")

# Show sample of transformed data
print(f"\nSample of data with new column names:")
print(df_renamed.head(3))

# Update the main dataframe
df = df_renamed
print(f"\nDataFrame successfully updated with snake_case column names!")

# Display some examples of the transformation
print(f"\n" + "="*60)
print("EXAMPLES OF TRANSFORMATIONS:")
examples = [
    ('Country.of.Origin', 'country_of_origin'),
    ('ICO.Number', 'ico_number'), 
    ('Total.Cup.Points', 'total_cup_points'),
    ('Category.One.Defects', 'category_one_defects'),
    ('Processing.Method', 'processing_method')
]

for old, expected in examples:
    actual = column_mapping.get(old, 'NOT FOUND')
    status = "✓" if actual == expected else "✗"
    print(f"{status} {old:<25} → {actual}")

=== COLUMN MANIPULATION TO SNAKE_CASE ===

ORIGINAL COLUMN NAMES:
 1. Species
 2. Owner
 3. Country.of.Origin
 4. Farm.Name
 5. Lot.Number
 6. Mill
 7. ICO.Number
 8. Company
 9. Altitude
10. Region
11. Producer
12. Number.of.Bags
13. Bag.Weight
14. In.Country.Partner
15. Harvest.Year
16. Grading.Date
17. Owner.1
18. Variety
19. Processing.Method
20. Aroma
21. Flavor
22. Aftertaste
23. Acidity
24. Body
25. Balance
26. Uniformity
27. Clean.Cup
28. Sweetness
29. Cupper.Points
30. Total.Cup.Points
31. Moisture
32. Category.One.Defects
33. Quakers
34. Color
35. Category.Two.Defects
36. Expiration
37. Certification.Body
38. Certification.Address
39. Certification.Contact
40. unit_of_measurement
41. altitude_low_meters
42. altitude_high_meters
43. altitude_mean_meters

Total columns: 43

COLUMN NAME TRANSFORMATION:
Species                   → species
Owner                     → owner
Country.of.Origin         → country_of_origin
Farm.Name                 → farm_name
Lot.Number               

### Problem 4. Bag weight (1 point)
What's up with the bag weights? Make all necessary changes to the column values. Don't forget to document your methods and assumptions.

In [5]:

# Continue from previous problems - assuming df is already loaded with snake_case columns
# df = pd.read_csv('merged_data_cleaned.csv', index_col=0)
# df.columns = [to_snake_case(col) for col in df.columns]  # from Problem 3

print("=== BAG WEIGHT ANALYSIS AND CLEANING ===\n")

# First, let's examine what's in the bag_weight column
print("ORIGINAL BAG WEIGHT ANALYSIS:")
print(f"Total records: {len(df)}")
print(f"Non-null bag weights: {df['bag_weight'].notna().sum()}")
print(f"Null bag weights: {df['bag_weight'].isna().sum()}")

# Show unique values and their counts
print(f"\nUnique bag weight values (top 20):")
bag_weight_counts = df['bag_weight'].value_counts()
print(bag_weight_counts.head(20))

print(f"\nTotal unique values: {len(bag_weight_counts)}")

# Analyze patterns
print(f"\n" + "="*60)
print("PATTERN ANALYSIS:")

# Check for different units and formats
kg_pattern = df['bag_weight'].astype(str).str.contains('kg', na=False)
lb_pattern = df['bag_weight'].astype(str).str.contains('lb', na=False)
numeric_only = df['bag_weight'].astype(str).str.match(r'^\d+\.?\d*$', na=False)

print(f"Values with 'kg': {kg_pattern.sum()}")
print(f"Values with 'lb'/'lbs': {lb_pattern.sum()}")  
print(f"Pure numeric (no unit): {numeric_only.sum()}")

# Show some problematic cases
print(f"\nProblematic/unusual values:")
unusual_values = df['bag_weight'].value_counts().tail(10)
print(unusual_values)

# Define cleaning function
def clean_bag_weight(weight_str):
    """
    Clean and standardize bag weight values.
    
    Assumptions and methods:
    1. Convert all weights to kilograms (kg) as the standard unit
    2. 1 lb = 0.453592 kg (standard conversion)
    3. Pure numeric values without units are assumed to be kg (most common pattern)
    4. Remove obvious data entry errors (unrealistic weights)
    5. Handle mixed units like "2 kg,lbs" by taking the first valid number and unit
    
    Returns: float value in kg, or NaN for invalid entries
    """
    
    if pd.isna(weight_str):
        return np.nan
    
    # Convert to string and clean
    weight_str = str(weight_str).strip().lower()
    
    # Handle empty strings
    if not weight_str or weight_str == 'nan':
        return np.nan
    
    # Extract numeric value and unit using regex
    # Look for patterns like: "60 kg", "100 lbs", "2", "2 kg,lbs"
    match = re.search(r'(\d+\.?\d*)\s*(kg|lb|lbs)?', weight_str)
    
    if not match:
        return np.nan
    
    numeric_value = float(match.group(1))
    unit = match.group(2) if match.group(2) else 'kg'  # Assume kg if no unit
    
    # Convert to kg
    if unit in ['lb', 'lbs']:
        weight_kg = numeric_value * 0.453592  # Convert pounds to kg
    else:  # kg or no unit (assumed kg)
        weight_kg = numeric_value
    
    # Data validation: remove unrealistic weights
    # Coffee bags typically range from 1 kg to 100 kg
    # Anything above 1000 kg is likely an error (total shipment weight, not individual bag)
    if weight_kg <= 0 or weight_kg > 1000:
        return np.nan
    
    return round(weight_kg, 2)

print(f"\n" + "="*60)
print("CLEANING PROCESS:")

# Apply cleaning function
print("Applying cleaning function...")
df['bag_weight_cleaned'] = df['bag_weight'].apply(clean_bag_weight)

# Show results
print(f"\nCLEANING RESULTS:")
print(f"Original non-null values: {df['bag_weight'].notna().sum()}")
print(f"Cleaned non-null values: {df['bag_weight_cleaned'].notna().sum()}")
print(f"Values removed as invalid: {df['bag_weight'].notna().sum() - df['bag_weight_cleaned'].notna().sum()}")

# Show before/after comparison for some examples
print(f"\nBEFORE/AFTER EXAMPLES:")
comparison_sample = df[['bag_weight', 'bag_weight_cleaned']].dropna(subset=['bag_weight']).head(15)
for idx, row in comparison_sample.iterrows():
    original = row['bag_weight']
    cleaned = row['bag_weight_cleaned']
    print(f"'{original}' → {cleaned} kg")

# Show distribution of cleaned weights
print(f"\n" + "="*60)
print("CLEANED WEIGHT DISTRIBUTION:")
print(df['bag_weight_cleaned'].describe())

print(f"\nMost common bag weights (kg):")
print(df['bag_weight_cleaned'].value_counts().head(10))

# Check for outliers that might need attention
print(f"\nPotential outliers (> 200 kg):")
outliers = df[df['bag_weight_cleaned'] > 200]['bag_weight_cleaned'].value_counts()
if len(outliers) > 0:
    print(outliers)
else:
    print("No outliers found")

# Final step: replace original column
print(f"\n" + "="*60)
print("FINAL STEP:")
print("Replacing original bag_weight column with cleaned version...")

df['bag_weight'] = df['bag_weight_cleaned']
df.drop('bag_weight_cleaned', axis=1, inplace=True)

print("✓ Bag weight column successfully cleaned and standardized to kg")

# Summary of changes made
print(f"\n" + "="*60)
print("DOCUMENTATION OF METHODS AND ASSUMPTIONS:")
print("""
1. STANDARDIZATION: All weights converted to kilograms (kg)

2. UNIT CONVERSIONS:
   - Pounds to kg: 1 lb = 0.453592 kg
   - Values with 'kg' kept as-is
   - Pure numeric values assumed to be kg (most common pattern)

3. DATA CLEANING:
   - Removed weights <= 0 kg (invalid)
   - Removed weights > 1000 kg (likely total shipment, not individual bag)
   - Handled mixed units like "2 kg,lbs" by taking first valid number

4. ASSUMPTIONS:
   - Coffee bags typically range 1-100 kg (industry standard)
   - Pure numeric values without units are kg (based on data pattern analysis)
   - Values > 1000 kg are data entry errors (total weight vs bag weight)

5. VALIDATION:
   - Rounded to 2 decimal places for consistency
   - Converted invalid entries to NaN for proper handling
""")

print(f"Final bag_weight column statistics:")
print(df['bag_weight'].describe())

=== BAG WEIGHT ANALYSIS AND CLEANING ===

ORIGINAL BAG WEIGHT ANALYSIS:
Total records: 1339
Non-null bag weights: 1339
Null bag weights: 0

Unique bag weight values (top 20):
bag_weight
1 kg       331
60 kg      256
69 kg      200
70 kg      156
2 kg       122
100 lbs     59
30 kg       29
5 lbs       23
6           19
20 kg       14
50 kg       14
10 kg       11
59 kg       10
1 lbs        8
1            7
3 lbs        7
5 kg         7
2 lbs        5
4 lbs        4
80 kg        4
Name: count, dtype: int64

Total unique values: 56

PATTERN ANALYSIS:
Values with 'kg': 1198
Values with 'lb'/'lbs': 116
Pure numeric (no unit): 27

Problematic/unusual values:
bag_weight
660 kg      1
1218 kg     1
2           1
18 kg       1
150 lbs     1
18000 kg    1
1 kg,lbs    1
132 lbs     1
34 kg       1
130 lbs     1
Name: count, dtype: int64

CLEANING PROCESS:
Applying cleaning function...

CLEANING RESULTS:
Original non-null values: 1339
Cleaned non-null values: 1321
Values removed as invalid: 18



### Problem 5. Dates (1 point)
This should remind you of problem 4 but it's slightly nastier. Fix the harvest years, document the process.

While you're here, fix the expiration dates, and grading dates. Unlike the other column, these should be dates (`pd.to_datetime()` is your friend).

In [9]:

# Continue from previous problems - assuming df is already loaded with snake_case columns
# df = pd.read_csv('merged_data_cleaned.csv', index_col=0)
# df.columns = [to_snake_case(col) for col in df.columns]  # from Problem 3

print("=== DATE COLUMNS CLEANING ===\n")

# First, let's examine all three date columns
date_columns = ['harvest_year', 'grading_date', 'expiration']

for col in date_columns:
    print(f"{col.upper()} ANALYSIS:")
    print(f"Total records: {len(df)}")
    print(f"Non-null values: {df[col].notna().sum()}")
    print(f"Null values: {df[col].isna().sum()}")
    print(f"Unique values: {df[col].nunique()}")
    print(f"Sample values:")
    print(df[col].value_counts().head(10))
    print("-" * 50)

print("\n" + "="*60)
print("1. CLEANING HARVEST YEAR")
print("="*60)

def clean_harvest_year(year_str):
    """
    Clean and standardize harvest year values.
    
    Methods and assumptions:
    1. Extract valid 4-digit years (1900-2025)
    2. Handle range formats like "2013/2014" by taking the first year
    3. Handle seasonal formats like "2017 / 2018" 
    4. Remove obviously invalid entries (non-year formats)
    5. Convert Spanish months and other text to NaN
    
    Returns: int year or NaN for invalid entries
    """
    
    if pd.isna(year_str):
        return np.nan
    
    year_str = str(year_str).strip()
    
    # Handle empty strings
    if not year_str or year_str.lower() == 'nan':
        return np.nan
    
    # Try to extract 4-digit year from various formats
    # Pattern 1: Simple 4-digit year (2012, 2013, etc.)
    simple_year_match = re.search(r'\b(19|20)\d{2}\b', year_str)
    if simple_year_match:
        year = int(simple_year_match.group())
        if 1900 <= year <= 2025:  # Reasonable range for coffee harvesting
            return year
    
    # Pattern 2: Range formats like "2013/2014", "2017 / 2018", "2010-2011"
    range_match = re.search(r'\b(19|20)\d{2}\s*[/\-]\s*(19|20)?\d{2}\b', year_str)
    if range_match:
        # Take the first year from the range
        first_year_match = re.search(r'\b(19|20)\d{2}\b', year_str)
        if first_year_match:
            year = int(first_year_match.group())
            if 1900 <= year <= 2025:
                return year
    
    # Pattern 3: Quarter formats like "4T/10", "4T/2010"
    quarter_match = re.search(r'4T/(\d{2,4})', year_str)
    if quarter_match:
        year_part = quarter_match.group(1)
        if len(year_part) == 2:
            # Assume 20XX for 2-digit years
            year = 2000 + int(year_part)
        else:
            year = int(year_part)
        if 1900 <= year <= 2025:
            return year
    
    # Pattern 4: Month year formats like "March 2010"
    month_year_match = re.search(r'\b(19|20)\d{2}\b', year_str)
    if month_year_match:
        year = int(month_year_match.group())
        if 1900 <= year <= 2025:
            return year
    
    # If no valid pattern found, return NaN
    return np.nan

# Apply harvest year cleaning
print("Cleaning harvest_year column...")
df['harvest_year_original'] = df['harvest_year'].copy()  # Keep original for comparison
df['harvest_year'] = df['harvest_year'].apply(clean_harvest_year)

print(f"HARVEST YEAR CLEANING RESULTS:")
print(f"Original non-null values: {df['harvest_year_original'].notna().sum()}")
print(f"Cleaned non-null values: {df['harvest_year'].notna().sum()}")
print(f"Values converted to NaN: {df['harvest_year_original'].notna().sum() - df['harvest_year'].notna().sum()}")

# Show before/after examples
print(f"\nBEFORE/AFTER EXAMPLES:")
comparison = df[['harvest_year_original', 'harvest_year']].dropna(subset=['harvest_year_original'])
problematic_examples = comparison[comparison['harvest_year_original'].astype(str).str.contains(r'[/\-]|T/|[a-zA-Z]', na=False)].head(10)
for idx, row in problematic_examples.iterrows():
    print(f"'{row['harvest_year_original']}' → {row['harvest_year']}")

print(f"\nCleaned harvest year distribution:")
print(df['harvest_year'].value_counts().sort_index().head(10))

print("\n" + "="*60)
print("2. CLEANING GRADING DATE")
print("="*60)

def clean_date_column(date_str, column_name):
    """
    Clean and convert date strings to pandas datetime objects.
    
    Methods:
    1. Handle format: "Month DDth, YYYY" (e.g., "April 4th, 2015")
    2. Handle format: "Month DDst/nd/rd/th, YYYY"
    3. Use pd.to_datetime with multiple format attempts
    4. Return NaT for invalid dates
    
    Returns: pandas datetime or NaT
    """
    
    if pd.isna(date_str):
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    if not date_str or date_str.lower() == 'nan':
        return pd.NaT
    
    try:
        # Try multiple approaches to parse the date
        
        # Method 1: Direct pd.to_datetime (handles many formats automatically)
        try:
            return pd.to_datetime(date_str, errors='raise')
        except:
            pass
        
        # Method 2: Handle ordinal suffixes (1st, 2nd, 3rd, 4th, etc.)
        # Remove ordinal suffixes: st, nd, rd, th
        cleaned_date = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
        try:
            return pd.to_datetime(cleaned_date, errors='raise')
        except:
            pass
        
        # Method 3: Try common date formats explicitly
        date_formats = [
            '%B %d, %Y',      # April 4, 2015
            '%b %d, %Y',      # Apr 4, 2015
            '%m/%d/%Y',       # 04/04/2015
            '%Y-%m-%d',       # 2015-04-04
            '%d/%m/%Y',       # 04/04/2015 (day/month/year)
        ]
        
        for fmt in date_formats:
            try:
                return pd.to_datetime(cleaned_date, format=fmt, errors='raise')
            except:
                continue
        
        # If all methods fail, return NaT
        return pd.NaT
        
    except Exception as e:
        return pd.NaT

# Clean grading date
print("Cleaning grading_date column...")
df['grading_date_original'] = df['grading_date'].copy()
df['grading_date'] = df['grading_date'].apply(lambda x: clean_date_column(x, 'grading_date'))

print(f"GRADING DATE CLEANING RESULTS:")
print(f"Original non-null values: {df['grading_date_original'].notna().sum()}")
print(f"Cleaned non-null values: {df['grading_date'].notna().sum()}")
print(f"Successfully converted: {df['grading_date'].notna().sum()}")
print(f"Failed conversions: {df['grading_date_original'].notna().sum() - df['grading_date'].notna().sum()}")

# Show examples
print(f"\nGRADING DATE EXAMPLES:")
grading_examples = df[['grading_date_original', 'grading_date']].dropna(subset=['grading_date']).head(10)
for idx, row in grading_examples.iterrows():
    print(f"'{row['grading_date_original']}' → {row['grading_date'].strftime('%Y-%m-%d')}")

print("\n" + "="*60)
print("3. CLEANING EXPIRATION DATE")
print("="*60)

# Clean expiration date
print("Cleaning expiration column...")
df['expiration_original'] = df['expiration'].copy()
df['expiration'] = df['expiration'].apply(lambda x: clean_date_column(x, 'expiration'))

print(f"EXPIRATION DATE CLEANING RESULTS:")
print(f"Original non-null values: {df['expiration_original'].notna().sum()}")
print(f"Cleaned non-null values: {df['expiration'].notna().sum()}")
print(f"Successfully converted: {df['expiration'].notna().sum()}")
print(f"Failed conversions: {df['expiration_original'].notna().sum() - df['expiration'].notna().sum()}")

# Show examples
print(f"\nEXPIRATION DATE EXAMPLES:")
expiration_examples = df[['expiration_original', 'expiration']].dropna(subset=['expiration']).head(10)
for idx, row in expiration_examples.iterrows():
    print(f"'{row['expiration_original']}' → {row['expiration'].strftime('%Y-%m-%d')}")

print("\n" + "="*60)
print("4. DATA VALIDATION AND SUMMARY")
print("="*60)

# Validate date relationships
print("DATE RELATIONSHIP VALIDATION:")

# Check if grading_date < expiration (should be true for valid data)
valid_relationships = df[(df['grading_date'].notna()) & (df['expiration'].notna())]
correct_order = (valid_relationships['grading_date'] < valid_relationships['expiration']).sum()
total_comparable = len(valid_relationships)

print(f"Records with both grading and expiration dates: {total_comparable}")
print(f"Records where grading < expiration: {correct_order}")
print(f"Records with date order issues: {total_comparable - correct_order}")

# Check harvest year vs grading date year
harvest_grading_compare = df[(df['harvest_year'].notna()) & (df['grading_date'].notna())].copy()
harvest_grading_compare['grading_year'] = harvest_grading_compare['grading_date'].dt.year
year_diff = harvest_grading_compare['grading_year'] - harvest_grading_compare['harvest_year']

print(f"\nHARVEST vs GRADING YEAR:")
print(f"Records with both harvest year and grading date: {len(harvest_grading_compare)}")
print(f"Average time from harvest to grading: {year_diff.mean():.1f} years")
print(f"Year difference distribution:")
print(year_diff.value_counts().sort_index().head(10))

# Clean up temporary columns
df.drop(['harvest_year_original', 'grading_date_original', 'expiration_original'], axis=1, inplace=True)

print("\n" + "="*60)
print("FINAL SUMMARY AND DOCUMENTATION")
print("="*60)

print(f"""
DATE CLEANING DOCUMENTATION:

1. HARVEST YEAR CLEANING:
   - Converted to integer years (int64)
   - Handled range formats (2013/2014 → 2013)
   - Handled seasonal formats (2017 / 2018 → 2017) 
   - Handled quarter formats (4T/10 → 2010)
   - Removed invalid text entries (Mayo a Julio → NaN)
   - Valid range: 1900-2025

2. GRADING DATE CLEANING:
   - Converted to pandas datetime objects
   - Handled ordinal suffixes (4th, 1st, 2nd, 3rd)
   - Primary format: "Month DDth, YYYY" 
   - Fallback to multiple date format attempts
   - Invalid dates converted to NaT

3. EXPIRATION DATE CLEANING:
   - Same approach as grading dates
   - Converted to pandas datetime objects
   - Consistent format handling

4. DATA TYPES AFTER CLEANING:
   - harvest_year: int64 (or float64 with NaN)
   - grading_date: datetime64[ns]
   - expiration: datetime64[ns]

5. VALIDATION:
   - Checked logical date relationships
   - Verified harvest-to-grading time patterns
   - Maintained data integrity
""")

print(f"Final data types:")
for col in ['harvest_year', 'grading_date', 'expiration']:
    print(f"{col}: {df[col].dtype}")

print(f"\nFinal non-null counts:")
for col in ['harvest_year', 'grading_date', 'expiration']:
    print(f"{col}: {df[col].notna().sum()}/{len(df)} ({df[col].notna().sum()/len(df)*100:.1f}%)")

=== DATE COLUMNS CLEANING ===

HARVEST_YEAR ANALYSIS:
Total records: 1339
Non-null values: 1292
Null values: 47
Unique values: 46
Sample values:
harvest_year
2012           354
2014           233
2013           181
2015           129
2016           124
2017            70
2013/2014       29
2015/2016       28
2011            26
2017 / 2018     19
Name: count, dtype: int64
--------------------------------------------------
GRADING_DATE ANALYSIS:
Total records: 1339
Non-null values: 1339
Null values: 0
Unique values: 567
Sample values:
grading_date
July 11th, 2012         25
December 26th, 2013     24
June 6th, 2012          19
August 30th, 2012       18
July 26th, 2012         15
October 8th, 2015       13
March 29th, 2013        13
September 27th, 2012    13
June 17th, 2010         12
October 20th, 2017      11
Name: count, dtype: int64
--------------------------------------------------
EXPIRATION ANALYSIS:
Total records: 1339
Non-null values: 1339
Null values: 0
Unique values: 566
Samp

### Problem 6. Countries (1 point)
How many coffees are there with unknown countries of origin? What can you do about them?

In [10]:
# Continue from previous problems - assuming df is already loaded with cleaned data
# df = pd.read_csv('merged_data_cleaned.csv', index_col=0)
# df.columns = [to_snake_case(col) for col in df.columns]  # from Problem 3

print("=== COUNTRIES ANALYSIS ===\n")

# Basic analysis of country_of_origin column
print("COUNTRY OF ORIGIN ANALYSIS:")
print(f"Total records: {len(df)}")
print(f"Non-null countries: {df['country_of_origin'].notna().sum()}")
print(f"Null/missing countries: {df['country_of_origin'].isna().sum()}")
print(f"Empty string countries: {(df['country_of_origin'] == '').sum()}")

# Calculate percentage of missing countries
missing_countries = df['country_of_origin'].isna().sum() + (df['country_of_origin'] == '').sum()
missing_percentage = (missing_countries / len(df)) * 100

print(f"\nMISSING COUNTRIES SUMMARY:")
print(f"Total missing: {missing_countries} ({missing_percentage:.2f}%)")

# Show country distribution
print(f"\nCOUNTRY DISTRIBUTION (top 15):")
country_counts = df['country_of_origin'].value_counts()
print(country_counts.head(15))

print(f"\nTotal unique countries: {df['country_of_origin'].nunique()}")

print("\n" + "="*60)
print("ANALYSIS OF MISSING COUNTRY RECORDS")
print("="*60)

# Get records with missing countries
missing_mask = df['country_of_origin'].isna() | (df['country_of_origin'] == '')
missing_records = df[missing_mask].copy()

print(f"Records with unknown countries: {len(missing_records)}")

if len(missing_records) > 0:
    print(f"\nDetailed analysis of missing country records:")
    
    # Show all relevant columns for missing records
    relevant_columns = ['owner', 'company', 'producer', 'region', 'farm_name', 'mill', 'ico_number', 
                       'harvest_year', 'grading_date', 'total_cup_points']
    
    print("\nMissing country records details:")
    for idx, row in missing_records.iterrows():
        print(f"\nRecord {idx}:")
        for col in relevant_columns:
            if col in df.columns:
                value = row[col]
                if pd.notna(value) and value != '':
                    print(f"  {col}: '{value}'")
                else:
                    print(f"  {col}: null/empty")

    print("\n" + "="*60)
    print("STRATEGIES FOR HANDLING MISSING COUNTRIES")
    print("="*60)

    print("STRATEGY 1: INFERENCE FROM OTHER FIELDS")
    print("-" * 40)
    
    # Try to infer from owner/company names
    def infer_country_from_text(text_fields):
        """
        Attempt to infer country from text fields using common patterns.
        
        This function looks for:
        1. Country-specific company suffixes (S.A., Ltd., etc.)
        2. Geographic indicators in names
        3. Known regional coffee organizations
        
        Returns: inferred country or None
        """
        
        if not text_fields or all(pd.isna(field) for field in text_fields):
            return None
        
        # Combine all text fields
        combined_text = ' '.join([str(field).lower() for field in text_fields if pd.notna(field)])
        
        # Country inference patterns
        country_patterns = {
            'mexico': ['mexican', 'mexico', 'mexicano', 'oaxaca', 'chiapas', 'veracruz'],
            'colombia': ['colombian', 'colombia', 'colombiano', 'huila', 'nariño', 'cauca'],
            'guatemala': ['guatemalan', 'guatemala', 'guatemalteco', 'antigua', 'huehuetenango'],
            'brazil': ['brazilian', 'brazil', 'brasileiro', 'minas gerais', 'sao paulo', 'bahia'],
            'costa rica': ['costa rica', 'costarricense', 'tarrazú', 'central valley'],
            'honduras': ['honduran', 'honduras', 'hondureño', 'copán', 'santa bárbara'],
            'ethiopia': ['ethiopian', 'ethiopia', 'yirgacheffe', 'sidamo', 'harrar'],
            'kenya': ['kenyan', 'kenya', 'aa', 'nyeri', 'kirinyaga'],
            'nicaragua': ['nicaraguan', 'nicaragua', 'nicaragüense', 'matagalpa', 'jinotega'],
            'panama': ['panamanian', 'panama', 'panameño', 'boquete', 'chiriquí'],
            'peru': ['peruvian', 'peru', 'peruano', 'chanchamayo', 'cusco'],
            'el salvador': ['salvadoran', 'el salvador', 'salvadoreño', 'santa ana', 'ahuachapán']
        }
        
        # Check for patterns
        for country, patterns in country_patterns.items():
            for pattern in patterns:
                if pattern in combined_text:
                    return country.title()
        
        # Check for company suffixes that might indicate country
        if 's.a.' in combined_text or 'sociedad anónima' in combined_text:
            # Could be Latin American country
            return None  # Too ambiguous
        
        if 'ltd' in combined_text or 'limited' in combined_text:
            # Could be English-speaking country, but not specific enough
            return None
        
        return None

    # Apply inference to missing records
    print("Attempting to infer countries from available data...")
    
    inferred_countries = []
    for idx, row in missing_records.iterrows():
        text_fields = [row.get('owner'), row.get('company'), row.get('producer'), 
                      row.get('region'), row.get('farm_name'), row.get('mill')]
        
        inferred = infer_country_from_text(text_fields)
        inferred_countries.append(inferred)
        
        print(f"\nRecord {idx}:")
        print(f"  Available info: {[field for field in text_fields if pd.notna(field) and field != '']}")
        print(f"  Inferred country: {inferred if inferred else 'Could not infer'}")

    # Strategy 2: ICO Number analysis
    print(f"\n" + "-" * 40)
    print("STRATEGY 2: ICO NUMBER ANALYSIS")
    print("-" * 40)
    
    # ICO numbers might contain country codes
    for idx, row in missing_records.iterrows():
        ico_number = row.get('ico_number')
        if pd.notna(ico_number) and ico_number != '':
            print(f"Record {idx} ICO Number: '{ico_number}'")
            
            # ICO numbers sometimes start with country codes or contain geographic info
            # This would require a lookup table of ICO country codes
            print(f"  Analysis: ICO number format suggests further research needed")

    print(f"\n" + "="*60)
    print("RECOMMENDED ACTIONS")
    print("="*60)
    
    actions_taken = []
    
    print("1. IMMEDIATE ACTIONS:")
    print("   - Document missing data: {} records ({:.2f}%)".format(missing_countries, missing_percentage))
    print("   - Keep records with missing countries (data is still valuable)")
    print("   - Flag records for manual review")
    
    print("\n2. RESEARCH ACTIONS:")
    print("   - Research ICO number patterns for geographic codes")
    print("   - Cross-reference owner/company names with coffee industry databases")
    print("   - Look up certification body addresses for geographic clues")
    
    print("\n3. DATA HANDLING OPTIONS:")
    
    # Option A: Create 'Unknown' category
    print("   Option A: Create 'Unknown' category")
    df_option_a = df.copy()
    df_option_a['country_of_origin'] = df_option_a['country_of_origin'].fillna('Unknown')
    df_option_a['country_of_origin'] = df_option_a['country_of_origin'].replace('', 'Unknown')
    
    print(f"     - Replace missing values with 'Unknown'")
    print(f"     - Preserves all records for analysis")
    print(f"     - Clear indication of missing data")
    
    # Option B: Remove records (not recommended for such few records)
    print("   Option B: Remove records with missing countries")
    print(f"     - Would remove {missing_countries} records")
    print(f"     - NOT RECOMMENDED: Loss of valuable quality data")
    
    # Option C: Statistical imputation (not appropriate for categorical country data)
    print("   Option C: Statistical imputation")
    print("     - NOT APPROPRIATE: Countries are nominal categorical data")
    print("     - No meaningful 'average' or 'most common' substitute")

else:
    print("✓ Great! No records with missing countries found.")

print(f"\n" + "="*60)
print("IMPLEMENTATION: HANDLING MISSING COUNTRIES")
print("="*60)

# Implement the recommended approach
print("Implementing Option A: Create 'Unknown' category for missing countries")

# Handle missing countries
original_nulls = df['country_of_origin'].isna().sum()
original_empty = (df['country_of_origin'] == '').sum()

# Replace missing values with 'Unknown'
df['country_of_origin'] = df['country_of_origin'].fillna('Unknown')
df['country_of_origin'] = df['country_of_origin'].replace('', 'Unknown')

# Verify changes
new_unknown = (df['country_of_origin'] == 'Unknown').sum()

print(f"RESULTS:")
print(f"- Original null values: {original_nulls}")
print(f"- Original empty strings: {original_empty}")
print(f"- New 'Unknown' entries: {new_unknown}")
print(f"- Total handled: {original_nulls + original_empty}")

# Final country distribution
print(f"\nFINAL COUNTRY DISTRIBUTION (top 10):")
final_country_counts = df['country_of_origin'].value_counts()
print(final_country_counts.head(10))

if new_unknown > 0:
    print(f"\n'Unknown' countries: {new_unknown} ({(new_unknown/len(df)*100):.2f}%)")

print(f"\n✓ Successfully handled missing country data")
print(f"✓ All {len(df)} records preserved")
print(f"✓ Missing data clearly marked as 'Unknown'")

print(f"\n" + "="*60)
print("SUMMARY AND DOCUMENTATION")
print("="*60)

print(f"""
MISSING COUNTRIES ANALYSIS SUMMARY:

1. FINDINGS:
   - Total records: {len(df)}
   - Missing countries: {original_nulls + original_empty} ({((original_nulls + original_empty)/len(df)*100):.2f}%)
   - Very low missing data rate - excellent data quality

2. ROOT CAUSE:
   - Appears to be isolated data entry issues
   - Not systematic missing data problem

3. SOLUTION IMPLEMENTED:
   - Replaced missing values with 'Unknown' category
   - Preserved all records (valuable quality data)
   - Clear documentation of missing data

4. ALTERNATIVE APPROACHES CONSIDERED:
   - Record removal: Rejected (unnecessary data loss)
   - Imputation: Rejected (inappropriate for categorical country data)
   - Manual research: Recommended for future data collection

5. IMPACT:
   - No data loss
   - Clear missing data handling
   - Ready for geographic analysis with proper missing data acknowledgment
""")

=== COUNTRIES ANALYSIS ===

COUNTRY OF ORIGIN ANALYSIS:
Total records: 1339
Non-null countries: 1338
Null/missing countries: 1
Empty string countries: 0

MISSING COUNTRIES SUMMARY:
Total missing: 1 (0.07%)

COUNTRY DISTRIBUTION (top 15):
country_of_origin
Mexico                          236
Colombia                        183
Guatemala                       181
Brazil                          132
Taiwan                           75
United States (Hawaii)           73
Honduras                         53
Costa Rica                       51
Ethiopia                         44
Tanzania, United Republic Of     40
Uganda                           36
Thailand                         32
Nicaragua                        26
Kenya                            25
El Salvador                      21
Name: count, dtype: int64

Total unique countries: 36

ANALYSIS OF MISSING COUNTRY RECORDS
Records with unknown countries: 1

Detailed analysis of missing country records:

Missing country records details

### Problem 7. Owners (1 point)
There are two suspicious columns, named `Owner`, and `Owner.1` (they're likely called something different after you solved problem 3). Do something about them. Is there any link to `Producer`?

### Problem 8. Coffee color by country and continent (1 point)
Create a table which shows how many coffees of each color are there in every country. Leave the missing values as they are.

**Note:** If you ask me, countries should be in rows, I prefer long tables much better than wide ones.

Now do the same for continents. You know what continent each country is located in.

### Problem 9. Ratings (1 point)
The columns `Aroma`, `Flavor`, etc., up to `Moisture` represent subjective ratings. Explore them. Show the means and range; draw histograms and / or boxplots as needed. You can even try correlations if you want. What's up with all those ratings?

### Problem 10. High-level errors (1 point)
Check the countries against region names, altitudes, and companies. Are there any discrepancies (e.g. human errors, like a region not matching the country)? Take a look at the (cleaned) altitudes; there has been a lot of preprocessing done to them. Was it done correctly?

### * Problem 11. Clean and explore at will
The dataset claimed to be clean, but we were able to discover a lot of things to fix and do better.

Play around with the data as much as you wish, and if you find variables to tidy up and clean - by all means, do that!